Notebook which focuses on the facial dataset and the performance comparison of algorithms on it

In [ ]:
from IPython.core.display import display, HTML
display(HTML('<style>.container {width:100% !important;}</style>'))

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt 
import numpy as np
import torch

import nmf.mult
import nmf.pgrad
import nmf.nesterov
import nmf.bayes

import nmf_torch.mult
import nmf_torch.pgrad
import nmf_torch.nesterov
import nmf_torch.norms

import matplotlib
import pickle

from theory.represent import rescale_WH

from read_data.reading import read_face_images, images_matrix_grid,\
                                roll_images, unroll_images 

from performance.performance_eval_func import get_random_lowrank_matrix, get_time_ratio,\
                              compare_performance, plot_performance_dict,\
                              torch_algo_wrapper, plot_errors_dict,\
                              plot_ratios_gpu_algo, plot_ratios_cpu_gpu, plot_ratios_cpu_algo,\
                              plot_differences_gpu_algo, plot_differences_cpu_gpu, plot_differences_cpu_algo

In [ ]:
image_data = read_face_images("data/att_faces/images").astype(float)

In [ ]:
print(image_data[0].shape, "{:.2f}".format(image_data.nbytes / 2**10), 
          "{:.2f}".format(image_data.nbytes / 2**20))

In [ ]:
grid = images_matrix_grid(image_data, (16, 25))
plt.imshow(grid, cmap="gray")

In [ ]:
unrolled_image_data, ori_shape = unroll_images(image_data)

In [ ]:
unrolled_image_data.shape

In [ ]:
algo_dict_to_test = {
    "mult": nmf.mult.factorise_Fnorm,
    "pgrad": nmf.pgrad.factorise_Fnorm_subproblems,
    "nesterov": nmf.nesterov.factorise_Fnorm,

    "mult_torch": torch_algo_wrapper(nmf_torch.mult.factorise_Fnorm, 
                                     device="cuda"),
    "pgrad_torch": torch_algo_wrapper(nmf_torch.pgrad.factorise_Fnorm_subproblems, 
                                      device="cuda"),
    "nesterov_torch": torch_algo_wrapper(nmf_torch.nesterov.factorise_Fnorm, 
                                        device="cuda")
}

In [ ]:
f, ax = plt.subplots()
plot_dict(errors_result, ax, log=False)

In [ ]:
errors_dict = pickle.load(open("faces_data_errors_dict.pkl", "rb"))

In [ ]:
inner_dim = 25

shape = unrolled_image_data.shape
W_init = np.random.rand(shape[0], inner_dim)
H_init = np.random.rand(inner_dim, shape[1])

In [ ]:
errors_dict = compare_performance(unrolled_image_data, 
                                      inner_dim, time_limit=100,
                                      W_init=W_init, H_init=H_init, 
                                      algo_dict_to_test=algo_dict_to_test)

In [ ]:
pickle.dump(errors_dict, open("faces_data_errors_dict.pkl", "wb"))

# Time ratio graphs

In [ ]:
gridspec_kw = dict(wspace=0.4, hspace=0.5,
                    top=0.85,
                    bottom=0.1,
                    left=0.1, right=0.95)

In [ ]:
shape = unrolled_image_data.shape
nmbytes = unrolled_image_data.nbytes / 2**20
inner_dim = 25

In [ ]:
shape = (400, 10304)
nmbytes = 31.45
inner_dim = 25

In [ ]:
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}

matplotlib.rc('font', **font)

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(10, 10), gridspec_kw=gridspec_kw)


f.suptitle("Facial image data represdented by {0} matrix, {2:.2f} MB \n Factorization of size {1}".format(shape, inner_dim,
                                                                       nmbytes))


plot_errors_dict(errors_dict, axes[0, 0], log=True, title="Cost function", x_lbl="time [s]")
plot_ratios_cpu_gpu(errors_dict, axes[0, 1])
plot_ratios_gpu_algo(errors_dict, axes[1, 0:], selected_algs=["mult_torch", "pgrad_torch", "nesterov_torch"])

axes[0, 1].set_title("CPU / GPU comparison")
axes[1, 0].set_title("multiplicative / gradient\nalgorithms comparison")
axes[1, 1].set_title("Nesterov / projected\n gradient algorithms comparison")


In [ ]:
nesterov = torch_algo_wrapper(nmf_torch.nesterov.factorise_Fnorm, device="cuda")
W, H = nesterov(V=unrolled_image_data, inner_dim=25, time_limit=40, max_steps=np.inf)

In [ ]:
W, H = rescale_WH(W, H)

In [ ]:
basis = roll_images(H, ori_shape)

f, axs = plt.subplots(2, 13, gridspec_kw=dict(hspace=0.01, wspace=0.05))
axs = axs.ravel()
for ax in axs:
    ax.axis('off')

f.suptitle("Basis")

for i in range(25):
    ax = axs[i]
    ax.imshow(basis[i], cmap="gray"); 

In [ ]:
f, [ax1, ax2] = plt.subplots(ncols=2, sharex=True, sharey=True)

approx_unrolled_image_data = W @ H
approx_image_data = roll_images(approx_unrolled_image_data, ori_shape)
grid = images_matrix_grid(approx_image_data, (25, 16))
ax1.imshow(grid, cmap="gray"); ax1.set_title("Approximated")
grid = images_matrix_grid(image_data, (25, 16))
ax2.imshow(grid, cmap="gray"); ax2.set_title("Original")

ax1.axis("off")
ax2.axis("off")

# Problem with Bayesian nonnegative matrix factorization

In [ ]:
W, H, errors_t = torch_algo_wrapper(nmf_torch.nesterov.factorise_Fnorm)(
                        V=unrolled_image_data,
                        inner_dim=25, 
                        max_steps=8000,
                        time_limit=5,
                        record_errors=True)

In [ ]:
W, H, errors = nmf.bayes.factorise_ICM(unrolled_image_data, 25, 
                                       max_steps=80000, time_limit=60, record_errors=True)

In [ ]:
image_components = roll_images(H, ori_shape)
grid = images_matrix_grid(image_components, (5, 5))
f, ax = plt.subplots()
ax.imshow(grid, cmap="gray")
ax.set_title("Image components")

The above image demontrates the lack of robustness of the bayesian nonnegative matrix factorisation algorithm